<h1> Get to know the project dataset, explore it, and prepare it for analysis </h1>

In this notebook, you will be working with a dataset about taxi rides in New York City. The dataset is hosted on Google Cloud's BigQuery serverless data warehouse. You will learn how to authenticate with Google Cloud from this Jupyter notebook enviroment and then use SQL and BigQuery APIs to retrieve a sample taxi fare dataset. Remember that your goal is to build a machine learning model that will predict taxi fares in New York City so that riders know approximately how much they will be charged before they take a cab. You will explore and clean up the dataset, prepare it for further processing, and will use Python along with libraries like Pandas and Seaborn to help you.


---
Before you start, **make sure that you are logged in with your student account**. Otherwise you may incur Google Cloud charges for using this notebook. 

---


To confirm that you are logged in with your student account, check that you see a letter S in a circle located in the upper right hand corner of this notebook like in the screenshot below. Of course the color around your letter S might not be exactly the same ;)

If you are not sure if you are logged in with your student account, close all your private (i.e. incognito/anonymous) windows, open a new one,  log in [here](https://console.cloud.google.com) using the student credentials you got earlier, and finally return back to this page to continue with the notebook.

![](https://i.imgur.com/PfSHVAb.png)

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns

import shutil
from google.cloud import bigquery

#@markdown Copy-paste your GCP Project ID in the following field:


PROJECT = "" #@param {type: "string"}

#@markdown Next, use Shift-Enter to run this cell and to complete authentication.

try:  
  from google.colab import auth
  auth.authenticate_user()  
  print("AUTHENTICATED")
except:
  print("FAILED to authenticate")

<h3> Sample data from BigQuery </h3>

The taxi dataset for this project is a BigQuery public dataset. To review the dataset schema you will need to open the link at the end of the next paragraph, accept the terms & conditions, select your GCP project by clicking on the "Select a project" dropdown as shown on the following screenshot, and refresh your page.

![](https://i.imgur.com/0sEOwMv.png)

After you click on the link at the end of this paragraph, take a look at the schema and the field names. Switch to the Details tab to verify that the number of rows is about 1.1  billion and then switch to the Preview tab to sample a few rows. Here's the link: <a href="https://console.cloud.google.com/bigquery?p=nyc-tlc&d=yellow&t=trips&page=table">NYC taxi fare dataset</a>



Here's a SQL query to sample 10 rows of data via BigQuery APIs. The SQL statement uses the LIMIT keyword to restrict the sample size to 10 rows. With this approach there are no guarantees about which records are returned or about their order. Notice that the code in the next cell is using BigQuery APIs and then stores the response in a Pandas dataframe variable named trips.

In [0]:
bq = bigquery.Client(project=PROJECT)

trips = bq.query('''
  SELECT 
    pickup_datetime, 
    pickup_longitude, 
    pickup_latitude, 
    dropoff_longitude,
    dropoff_latitude, 
    passenger_count, 
    trip_distance, 
    tolls_amount, 
    fare_amount,
    total_amount 
  FROM `nyc-tlc.yellow.trips` 
  LIMIT 10
  ''').to_dataframe()

trips

Let's increase the number of records and do some neat graphs.  To properly sample the dataset, hash the pickup time and return 1 in 100,000 records. Since there are roughly 1 billion records in the data, you should get back approximately 10,000 records.

In [0]:
EVERY_N = 100000

trips = bq.query('''
  SELECT
    pickup_datetime,
    pickup_longitude, pickup_latitude, 
    dropoff_longitude, dropoff_latitude,
    passenger_count,
    trip_distance,
    tolls_amount,
    fare_amount,
    total_amount
  FROM
    `nyc-tlc.yellow.trips`
  WHERE
    #notice that the string (bytes) of the pickup_datetime
    #are hashed to INT64 and then converted to its absolute (positive) value.
    #The positive INT64 value is then divided by EVERY_N and
    #only the rows where the remainder is 1 are returned
    MOD(ABS(FARM_FINGERPRINT(STRING(pickup_datetime))), %d) = 1
''' % (EVERY_N)).to_dataframe()

trips[:10]

<h3> Exploring data </h3>

Let's explore this dataset and clean it up as necessary. The following cells use Python Seaborn package to visualize graphs and Pandas to do the slicing and filtering of data.

In [0]:
ax = sns.regplot(x="trip_distance", y="fare_amount", fit_reg=False, ci=None, truncate=True, data=trips)

Hmm ... do you see something wrong with the data that needs addressing?

It appears that there is bogus data that is being coded as zero distance and some fare amounts that are definitely illegitimate. Let's remove them from our analysis. You can do this by modifying the SQL statement to keep just the trips that are longer than zero miles and with fare amounts that are at least the minimum cab fare ($2.50).

Note the extra WHERE clauses.

In [0]:
EVERY_N = 100000

trips = bq.query('''
  SELECT
    pickup_datetime,
    pickup_longitude, pickup_latitude, 
    dropoff_longitude, dropoff_latitude,
    passenger_count,
    trip_distance,
    tolls_amount,
    fare_amount,
    total_amount
  FROM
    `nyc-tlc.yellow.trips`
  WHERE
    MOD(ABS(FARM_FINGERPRINT(STRING(pickup_datetime))), %d) = 1
    
    #note that that trips with zero distance or 
    #costing less than $2.50 are excluded    
    AND trip_distance > 0 AND fare_amount >= 2.5    
    
''' % (EVERY_N)).to_dataframe()

ax = sns.regplot(x="trip_distance", y="fare_amount", fit_reg=False, ci=None, truncate=True, data=trips)

What's up with the streaks at `$45` and `$50`?  Those are fixed-amount rides from JFK and La Guardia airports into anywhere in Manhattan, i.e. to be expected.

Let's examine whether the toll amount is captured in the total amount.

In [0]:
tollrides = trips[trips['tolls_amount'] > 0]
tollrides[['tolls_amount', 'fare_amount',	'total_amount']][:5]

Looking a few samples above, it should be clear that the total amount reflects fare amount, toll and tip somewhat arbitrarily -- this is because when customers pay cash, the tip is not known.  So,  the sum of fare_amount + tolls_amount is what needs to be predicted.  Tips are discretionary and do not need to be included for fare estimation.

Let's also look at the distribution of values within the columns.

In [0]:
trips.describe()

Hmm ... The min, max of locations look strange. We shouldn't have zero values for location!

<h3> Quality control and other preprocessing </h3>

Some clean-up of the data is in order:
<ol>
<li>New York city longitudes are around -74 and latitudes are around 41.</li>
<li>There shouldn't be zero passengers.</li>
<li>Clean up the total_amount column to reflect only fare_amount and tolls_amount, and then remove those two columns.</li>
<li>Before the ride starts, the pickup and dropoff locations are known but not the actual trip distance (that depends on the route taken), so remove it from the ML dataset</li>
<li>Discard the timestamp</li>
</ol>

We could do this kind of preprocessing in BigQuery, similar to how we removed the zero-distance rides, but just to show you another option, let's do this in Python.  In production, we'll have to carry out the same preprocessing on the real-time input data. 

This sort of preprocessing of input data is quite common in ML, especially if the quality-control is dynamic.

In [0]:
def preprocess(trips_in):
  trips = trips_in.copy(deep=True)
  trips.fare_amount = trips.fare_amount + trips.tolls_amount
  del trips['tolls_amount']
  del trips['total_amount']
  del trips['trip_distance']
  del trips['pickup_datetime']
  qc = np.all([\
             trips['pickup_longitude'] > -78, \
             trips['pickup_longitude'] < -70, \
             trips['dropoff_longitude'] > -78, \
             trips['dropoff_longitude'] < -70, \
             trips['pickup_latitude'] > 37, \
             trips['pickup_latitude'] < 45, \
             trips['dropoff_latitude'] > 37, \
             trips['dropoff_latitude'] < 45, \
             trips['passenger_count'] > 0,
            ], axis=0)
  return trips[qc]

tripsqc = preprocess(trips)
tripsqc.describe()

The quality control has removed about 300 rows (10700 - 10400) or about 3% of the data. This seems reasonable.

In the next lab, you will move on to creating the ML datasets.

<h3> Recap </h3>

In this notebook, you used BigQuery APIs to sample data about taxi rides in New York City. Based on what you know about using hashing functions to retrieve reproducible samples, you practiced with SQL statements that have been implemented to use the FARM_FINGERPRINT hashing function to selectively and consistently filter out specific rows of data. Finally, you discovered bogus data and removed it from analysis using both SQL and Python based implementations.

Copyright 2019 Counter Factual .AI LLC.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.